In [177]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [178]:
fr = pd.read_csv('../data/search/CD31/validation/Final_Results.csv')

ap = pd.read_excel(r'..\data\metadata\animal_phenotypes.xlsx', index_col=0)
apval = pd.read_excel(r'../data/metadata/animal_phenotypes_validation.xlsx', index_col=0)

fg = pd.read_csv(r'..\data\metadata\combined_metab_lipid_file_grouping.csv', index_col=0)

### Validate OGTT AUC calculation, write correct value 

In [45]:
# ogtt_timepoints = apval.filter(regex='OGTT \(').columns
# for i, row in apval.iterrows():
#     try: 
#         trap = np.trapz(y=row[ogtt_timepoints].values, x=[0, 10, 20, 30, 60, 120])
        
#         print(trap, row['OGTT(AUC)'])
#         apval.loc[i, 'OGTT(AUC)'] = trap
#         print(trap == row['OGTT(AUC)'] )
#     except:
#         print(i)
#         pass
# apval.to_excel(r'../data/metadata/animal_phenotypes_validation.xlsx')

## Make file grouping df

In [89]:
# amd = Animal MetaData
sample_data = pd.read_excel('../data/metadata/second round plasma samples.xlsx')
sample_data.index = sample_data['animal'].astype('str') + '_' + sample_data['week'].astype('str') + '_RBG'
sample_data

,rat,animal,week,color,date,batch,include,sex,Huishi note,Ben note
1061_8_RBG,1061M,1061,8,slightly pink,20220325,1,True,M,NaN,NaN
1061_9_RBG,1061M,1061,9,clear,20220401,1,True,M,NaN,NaN
1061_10_RBG,1061M,1061,10,slightly pink,20220408,1,True,M,NaN,NaN
1075_8_RBG,1075M,1075,8,slightly pink,20220325,1,True,M,NaN,NaN
1075_9_RBG,1075M,1075,9,clear,20220401,1,True,M,NaN,NaN
1075_10_RBG,1075M,1075,10,clear,20220408,1,True,M,NaN,NaN
1081_8_RBG,1081M,1081,8,clear,20220325,1,True,M,NaN,NaN
1081_9_RBG,1081M,1081,9,clear,20220401,1,True,M,NaN,NaN
1081_10_RBG,1081M,1081,10,clear,20220408,1,True,M,NaN,NaN
1094_8_RBG,1094M,1094,8,pink,20220330,1,True,M,NaN,NaN


In [120]:
file_metadata = {}
splits = {x: x.split('.')[0].split('_') for x in fr.filter(regex='.raw').columns}
for file, split in splits.items():
    print(split)
    d = file_metadata[file] = {}
    if '1700s' in split or '1000s' in split:
        continue
#         d['cohort'] = split[2]
#         if 'blank' in split:
#             d['blank'] = True
#         else:
#             d['blank'] = False
#         if 'dil' in split:
#             d['QC_dilution'] = True
#         else:
#             d['QC_dilution'] = False
#         if 'QC' in split[3] and 'rep' in split[4]:
#             d['QC_rep'] = True
#         else:
#             d['QC_rep'] = False
    else:
        if split[2][:2] == '10':
            d['cohort'] = '1000s'
        else:
            d['cohort'] = '1700s'
        d['animal'] = split[2][:-1]
        d['week'] = split[3][:-2]
        

['BJA-Nilerat-20220915', 'lipid', '1700s', 'extract', 'blank', 'rep1']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'extract', 'blank', 'rep2']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'extract', 'blank', 'rep3']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC25uL', 'dil']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC50uL', 'dil']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'dil']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep1']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep2']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep3']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep4']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep5']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep6']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep7']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC100uL', 'rep8']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC150uL', 'dil']
['BJA-Nilerat-20220915', 'lipid', '1700s', 'QC200

In [170]:
apval['OGTT(AUC)']

1727    38202.5
1728    43767.5
1731    41218.5
1732    37860.0
1735    36090.0
1741    44045.0
1745        NaN
1746    29785.0
1748    37650.0
1755        NaN
1742        NaN
1743        NaN
1754        NaN
1788        NaN
1789        NaN
1798        NaN
1809        NaN
1810        NaN
1811        NaN
1812        NaN
1857        NaN
Name: OGTT(AUC), dtype: float64

In [171]:
fgval['animal'].map(apval['OGTT(AUC)'])

1061_10_RBG        NaN
1061_8_RBG         NaN
1061_9_RBG         NaN
1075_10_RBG        NaN
1075_8_RBG         NaN
1075_9_RBG         NaN
1081_10_RBG        NaN
1081_8_RBG         NaN
1081_9_RBG         NaN
1094_10_RBG        NaN
1094_8_RBG         NaN
1094_9_RBG         NaN
1095_10_RBG        NaN
1095_8_RBG         NaN
1095_9_RBG         NaN
1096_10_RBG        NaN
1096_8_RBG         NaN
1096_9_RBG         NaN
1727_27_RBG    38202.5
1727_28_RBG    38202.5
1727_29_RBG    38202.5
1728_27_RBG    43767.5
1728_28_RBG    43767.5
1728_30_RBG    43767.5
1731_30_RBG    41218.5
1731_31_RBG    41218.5
1731_32_RBG    41218.5
1732_27_RBG    37860.0
1732_28_RBG    37860.0
1732_29_RBG    37860.0
1735_29_RBG    36090.0
1735_30_RBG    36090.0
1735_31_RBG    36090.0
1741_26_RBG    44045.0
1741_27_RBG    44045.0
1741_28_RBG    44045.0
1742_32_RBG        NaN
1742_33_RBG        NaN
1742_34_RBG        NaN
1743_31_RBG        NaN
1743_32_RBG        NaN
1743_33_RBG        NaN
1746_30_RBG    29785.0
1746_31_RBG

In [173]:
fgval = pd.DataFrame(file_metadata).T#.replace(np.nan, False)
fgval['fr_name'] = fgval.index
fgval.index = fgval['animal'].astype('str') + '_' + fgval['week'].astype('str') + '_RBG'
fgval = sample_data.join(fgval, lsuffix='_l', rsuffix='_r')
fgval = fgval.drop(fgval.loc[fgval['cohort'].isna()].index)
fgval = fgval.drop(['rat', 'animal_r', 'week_r', 'batch', 'include', 'Huishi note', 'Ben note'], axis=1)
fgval.rename({'animal_l': 'animal', 'week_l': 'week'}, axis=1, inplace=True)
fgval['date'] = pd.to_datetime(fgval['date'], format='%Y%m%d')
fgval['ogtt'] = fgval['animal'].map(apval['OGTT(AUC)'])
# fgval.to_csv('../data/metadata/file_grouping_validation.csv')
fgval

,animal,week,color,date,sex,cohort,fr_name,ogtt
1061_10_RBG,1061,10,slightly pink,2022-04-08,M,1000s,BJA-Nilerat-20220919_lipid_1061M_10wk_0408.raw...,NaN
1061_8_RBG,1061,8,slightly pink,2022-03-25,M,1000s,BJA-Nilerat-20220919_lipid_1061M_8wk_0325.raw ...,NaN
1061_9_RBG,1061,9,clear,2022-04-01,M,1000s,BJA-Nilerat-20220919_lipid_1061M_9wk_0401.raw ...,NaN
1075_10_RBG,1075,10,clear,2022-04-08,M,1000s,BJA-Nilerat-20220919_lipid_1075M_10wk_0408.raw...,NaN
1075_8_RBG,1075,8,slightly pink,2022-03-25,M,1000s,BJA-Nilerat-20220919_lipid_1075M_8wk_0325.raw ...,NaN
1075_9_RBG,1075,9,clear,2022-04-01,M,1000s,BJA-Nilerat-20220919_lipid_1075M_9wk_0401.raw ...,NaN
1081_10_RBG,1081,10,clear,2022-04-08,M,1000s,BJA-Nilerat-20220919_lipid_1081M_10wk_0408.raw...,NaN
1081_8_RBG,1081,8,clear,2022-03-25,M,1000s,BJA-Nilerat-20220919_lipid_1081M_8wk_0325.raw ...,NaN
1081_9_RBG,1081,9,clear,2022-04-01,M,1000s,BJA-Nilerat-20220919_lipid_1081M_9wk_0401.raw ...,NaN
1094_10_RBG,1094,10,clear,2022-04-13,M,1000s,BJA-Nilerat-20220919_lipid_1094M_10wk_0413.raw...,NaN


## Replace fr column names for ease of processing

In [175]:
combined_col_names = {fr_name: combined_name for fr_name, combined_name in zip(fgval['fr_name'], fgval.index)}
fr.rename(combined_col_names, axis=1, inplace=True)
val_data_cols = list(combined_col_names.values())
df = fr.loc[:, fgval.loc[fgval['animal'].isin([1727, 1728, 1731, 1732, 1735, 1741, 1746, 1748])].index]

In [176]:
df

,1727_27_RBG,1727_28_RBG,1727_29_RBG,1728_27_RBG,1728_28_RBG,1728_30_RBG,1731_30_RBG,1731_31_RBG,1731_32_RBG,1732_27_RBG,...,1735_31_RBG,1741_26_RBG,1741_27_RBG,1741_28_RBG,1746_30_RBG,1746_31_RBG,1746_32_RBG,1748_29_RBG,1748_30_RBG,1748_31_RBG
0,1.064489e+06,1.328877e+06,7.487449e+05,5.429591e+05,1.007532e+06,8.854894e+05,1.283011e+06,1.313621e+06,1.932653e+06,1.558176e+06,...,8.898480e+05,8.717958e+05,9.346366e+05,7.362930e+05,9.443548e+05,8.895936e+05,1.611157e+06,1.018463e+06,1.598290e+06,9.370545e+05
1,8.284636e+05,9.842930e+05,7.419131e+05,1.002276e+07,6.353520e+06,9.941596e+05,8.512516e+05,7.780155e+06,1.223018e+06,1.832529e+06,...,8.430247e+06,1.153668e+06,1.410869e+06,5.644427e+06,6.687586e+06,9.715960e+06,9.124779e+06,1.703321e+06,7.038349e+05,1.394914e+06
2,1.054155e+06,2.014870e+06,1.860647e+06,2.160098e+06,1.087097e+06,7.787837e+05,3.909059e+05,7.595526e+05,9.227629e+05,5.605037e+05,...,7.928404e+05,5.405122e+05,4.936062e+05,2.788401e+06,9.645712e+05,1.191310e+06,1.583347e+06,1.964838e+06,3.057224e+06,2.945574e+06
3,3.070528e+07,1.182106e+07,6.406309e+07,6.811908e+07,1.438239e+07,2.612402e+07,9.909235e+06,2.072455e+07,1.171061e+07,8.550390e+06,...,4.976819e+07,4.664097e+07,3.446278e+07,5.756702e+07,3.559271e+07,4.289665e+07,4.728058e+07,3.557606e+07,5.077748e+07,4.656953e+07
4,1.567894e+06,1.911106e+06,2.375974e+06,2.434296e+06,2.014958e+06,2.295790e+06,2.005742e+06,3.063410e+06,2.118355e+06,1.916806e+06,...,3.579002e+06,1.726821e+06,1.640991e+06,2.180931e+06,1.853755e+06,1.433279e+06,4.276774e+06,2.858830e+06,2.672690e+06,2.926361e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6501,1.170759e+05,1.186721e+05,9.913846e+04,9.686913e+04,1.168429e+05,1.072125e+05,1.107461e+05,1.084450e+05,1.009319e+05,1.012519e+05,...,1.161452e+05,1.175583e+05,1.047178e+05,1.134754e+05,9.482362e+04,1.160765e+05,1.041880e+05,1.077191e+05,1.090400e+05,9.962858e+04
6502,1.230077e+05,1.246848e+05,1.041614e+05,1.017771e+05,1.227629e+05,1.126445e+05,1.163572e+05,1.139395e+05,1.060457e+05,1.063820e+05,...,1.220298e+05,1.235145e+05,1.100234e+05,1.192247e+05,9.962795e+04,1.219576e+05,1.094668e+05,1.131768e+05,1.145646e+05,1.046764e+05
6503,1.091380e+05,1.106260e+05,9.241672e+04,9.030126e+04,1.089208e+05,9.994335e+04,1.032374e+05,1.010923e+05,9.408854e+04,9.438691e+04,...,1.082704e+05,1.095877e+05,9.761777e+04,1.057816e+05,8.839444e+04,1.082063e+05,9.712393e+04,1.004156e+05,1.016469e+05,9.287361e+04
6504,1.666631e+05,1.689354e+05,1.411283e+05,1.378978e+05,1.663314e+05,1.526221e+05,1.576524e+05,1.543766e+05,1.436813e+05,1.441369e+05,...,1.653382e+05,1.673498e+05,1.490707e+05,1.615375e+05,1.349859e+05,1.652404e+05,1.483166e+05,1.533432e+05,1.552236e+05,1.418260e+05


In [162]:
apval

,sex,cohort,note,OGTT_date,OGTT_age,OGTT_weight,OGTT_dextrose,OGTT (0 min),OGTT (10 min),OGTT (20 min),...,Weightweek25,Weightweek26,Weightweek27,Weightweek28,Weightweek29,Weightweek30,Weightweek31,Weightweek32,Weightweek33,Weightweek34
1727,M,1700s,NaN,2022-07-08,31.0,117.0,275.0,105.0,135.0,218.0,...,114.0,114.0,116.0,116.0,116.0,115.0,115.0,117.0,NaN,NaN
1728,M,1700s,missing 29wk plasma,2022-07-08,31.0,123.0,289.0,120.5,221.0,306.5,...,123.0,123.0,125.0,126.0,126.0,125.0,125.0,126.0,NaN,NaN
1731,F,1700s,OGTT AUC higher than expected,2022-07-29,34.0,111.0,261.0,67.0,143.5,316.0,...,100.0,103.0,105.0,105.0,107.0,110.0,110.0,111.0,110.0,111.0
1732,M,1700s,NaN,2022-07-15,32.0,115.0,270.0,96.5,147.0,236.5,...,112.0,113.0,113.0,114.0,112.0,116.0,116.0,NaN,NaN,NaN
1735,M,1700s,NaN,2022-07-22,33.0,121.0,284.0,73.0,137.5,301.5,...,115.0,115.0,117.0,118.0,117.0,118.0,118.0,120.0,121.0,NaN
1741,M,1700s,OGTT (FBG sample only 6uL) so we recollected a...,2022-07-15,32.0,106.0,249.0,63.5,244.5,345.0,...,106.0,105.0,106.0,107.0,109.0,109.0,NaN,NaN,NaN,NaN
1745,M,1700s,died wk 30 no OGTT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,124.0,124.0,124.0,124.0,124.0,121.0,NaN,NaN,NaN,NaN
1746,F,1700s,NaN,2022-07-29,34.0,123.0,289.0,81.0,135.0,194.5,...,114.0,116.0,116.0,116.0,118.0,121.0,121.0,121.0,121.0,121.0
1748,F,1700s,NaN,2022-07-29,33.0,129.0,303.0,66.5,210.5,219.0,...,123.0,123.0,123.0,123.0,126.0,127.0,128.0,129.0,129.0,NaN
1755,F,1700s,euthanized no OGTT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,110.0,113.0,114.0,118.0,119.0,120.0,121.0,121.0,NaN,NaN


# Draw quant vs ogtt 

In [4]:
top_feat_ids = [
    'CE 18:1', 
    'Plasmanyl-PC O-20:0_20:4', 
    'FA 18:0',
    'PC 36:3', 'PC 38:7', 
    '4-Guanidinobutyric acid', 
    'SM d37:1',
    'TG 20:5_22:6_22:6', 'TG 22:6_22:6_22:6', 'TG 18:1_22:6_22:6',
]
df = fr.loc[fr['Identification'].isin(top_feat_ids)]
df

,Retention Time (min),Quant Ion,Polarity,Area (max),Identification,Lipid Class,Features Found,BJA-Nilerat-20220915_lipid_1700s_extract_blank_rep1.raw (F1),BJA-Nilerat-20220915_lipid_1700s_extract_blank_rep2.raw (F2),BJA-Nilerat-20220915_lipid_1700s_extract_blank_rep3.raw (F3),...,BJA-Nilerat-20220919_lipid_1094M_8wk_0330.raw (F90),BJA-Nilerat-20220919_lipid_1094M_9wk_0406.raw (F91),BJA-Nilerat-20220919_lipid_1094M_10wk_0413.raw (F92),BJA-Nilerat-20220919_lipid_1095M_8wk_0330.raw (F93),BJA-Nilerat-20220919_lipid_1095M_9wk_0406.raw (F94),BJA-Nilerat-20220919_lipid_1095M_10wk_0413.raw (F95),BJA-Nilerat-20220919_lipid_1096M_8wk_0330.raw (F96),BJA-Nilerat-20220919_lipid_1096M_9wk_0406.raw (F97),BJA-Nilerat-20220919_lipid_1096M_10wk_0413.raw (F98),Unnamed: 105
1830,7.412,804.55878,+,4.202681e+07,PC 38:7,PC,15,172742.34090,174093.50350,1.785982e+05,...,2.806782e+07,504270.55720,1.495603e+06,1.805444e+07,2.670153e+07,309564.24980,3.505552e+07,1.417278e+07,7.746720e+05,NaN
1966,7.753,804.56104,+,6.177524e+06,PC 38:7,PC,18,51515.00390,50697.39250,5.146560e+04,...,5.001948e+06,58328.97168,1.776296e+05,3.662393e+06,4.622807e+06,58096.55610,4.954397e+06,3.378414e+06,7.870905e+04,NaN
1996,7.848,784.59149,+,3.160803e+07,PC 36:3,PC,4,71314.30066,57327.57288,5.987112e+04,...,2.330748e+07,45749.20287,1.158506e+06,1.082980e+07,1.527338e+07,44919.48749,3.160803e+07,7.011587e+06,2.990482e+05,NaN
2089,8.060,784.59155,+,2.205637e+08,PC 36:3,PC,27,430363.34420,172399.28660,1.342851e+05,...,1.898033e+08,235153.96100,8.549757e+06,1.465078e+08,1.251944e+08,428826.17310,2.078092e+08,6.353069e+07,2.773730e+06,NaN
2110,8.103,804.56097,+,1.236981e+08,PC 38:7,PC,72,234683.34040,384413.93020,1.422522e+06,...,3.295705e+07,397195.90620,4.344429e+06,1.148880e+07,1.616569e+07,319218.81910,2.255380e+07,2.020776e+07,1.498752e+06,NaN
2159,8.166,784.59210,+,9.571763e+07,PC 36:3,PC,19,219587.62480,186873.77340,1.671300e+05,...,7.545796e+07,233501.84740,3.425415e+06,2.618203e+07,6.196366e+07,151492.19890,8.846536e+07,3.201345e+07,1.263573e+06,NaN
2201,8.289,804.55865,+,5.261131e+07,PC 38:7,PC,8,52484.00067,78718.94746,9.092413e+04,...,1.881745e+07,58325.02950,1.053213e+06,6.713301e+06,1.309190e+07,54651.77083,1.835830e+07,1.087194e+07,8.850293e+04,NaN
2289,8.465,804.56122,+,2.267368e+07,PC 38:7,PC,2,50636.82377,59044.35009,6.330704e+04,...,1.881745e+07,46060.36090,1.053213e+06,1.994568e+06,1.216429e+07,53044.73137,1.450252e+07,2.713856e+06,4.752031e+04,NaN
3140,9.928,784.59265,+,5.171799e+07,PC 36:3,PC,10,220582.41820,235140.19670,2.731436e+05,...,2.529287e+07,670865.18110,1.547110e+06,1.758259e+07,2.023647e+07,572664.30000,2.973739e+07,2.056145e+07,5.887614e+05,NaN
3545,10.616,745.62909,+,4.139042e+07,SM d37:1,SM,125,83837.55841,69188.48669,1.052389e+05,...,1.193218e+07,111439.46420,8.039078e+05,6.689442e+06,6.794548e+06,167664.11890,9.109714e+06,8.117885e+06,1.930039e+05,NaN


## De-duplicate

In [ ]:
df